# Homework 4
<p> Thomas Dougherty <br>
10-14-2023 </p>

## Problem 1

`What is transfer learning? Look it up, and write how you understood it.`
<p> Transfer learning is the act of training a model for one task (taking a pretrained model) and fine tuning it for another specific task. Transfer learning is a more efficient way of training neural networks because it allows them to use knowledge from the previous task while working with less labeled data. </br>

It's like going from a riding a mountain bike to learning how to ride a motorized dirt bike. All the time spent on a mountain bike taught you balance, cornering technique, and riding over rough terrain. Since those previous skills don't have to br learned again, you can focus on things like throttle control and riding at high speed giving you a headstart on the learning process. </p> </br>

`Explain where the differences are that make the loss plot different between the first two notebooks.`
1. neaclass2 is using a pretrained resnet18 model whereas neaclass1 is using a newly instantiated resnet18 model. This gives it a head start in the training process.
</br>
2. neaclass2 has learning rate scheduler. By slowing incrementally slowing down the learning rate over a period of epochs, the model will find the simple patterns first then "zero in" on finding complex patterns.
</br>

`Finally, run neaclass3.ipynb  and explain the differences with respect to neoclass2.ipynb. At the end of neaclass3.ipynb create a test dataset from  NEUdata_split/Test and  use it to evaluate the accuracy of both its models.`
1. neaclass3 deepcopies the original pre-trained resnet18 model.
2. neaclass3 
<br>
Accuracy for original Resnet18 model came out to 95%, whereas accuracy for the vector model came out to around 25%

In [1]:
import os
#from utils import DatasetUtils
import torch
from torch import optim 
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from PIL import Image
from torchvision import transforms
from torchvision.datasets import DatasetFolder
import cv2
import torch.nn as nn
import numpy as np

!git clone https://github.com/skokalj/RowanDLclassNEA.git

fatal: destination path 'RowanDLclassNEA' already exists and is not an empty directory.


## Problem 2
### Design a convolutional autoencoder for a dataset of images 3 x 224 x224. Train it on NEU data for 50 epochs.

I'll start with getting the testing & training data and transforms

In [2]:
def load_image(img_path:str):    
        np_img = cv2.imread(img_path)
        return Image.fromarray(np_img)
"""
test_loader = DatasetUtils.create_loader_and_transform(
    root_path='RowanDLclassNEA/NEUdata_split/Test', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=32,
    shuffle=False,
    is_test=True
)

train_loader, val_loader, train_set, val_set = DatasetUtils.create_loader_and_transform(
    root_path='NEUdata', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=16,
    shuffle=True,
    is_test=False
)

"""
dset = DatasetFolder(root='RowanDLclassNEA/NEUdata', loader = load_image, extensions = ('.bmp',))

transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])


test_xform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])

dset = DatasetFolder(root='RowanDLclassNEA/NEUdata', loader = load_image, extensions = ('.bmp',), transform = transform)
test_data = DatasetFolder(root='RowanDLclassNEA/NEUdata_split/Test', loader = load_image, extensions=('.bmp'), transform=test_xform)


train_set, val_set = random_split(
                      dset, 
                      [1200, 600])

train_loader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=16, 
                    shuffle=True)

val_loader = torch.utils.data.DataLoader(
                    val_set, 
                    batch_size=16, 
                    shuffle=True)


test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size = 16,
            shuffle = False)

print(len(train_loader))
print(len(val_loader))
#print(len(test_loader))
print(train_set)


75
38


### Autoencorder Class

Found that pooling layers were needed for this just to bring the memory usage down to a reasonable level. I experimented with unpooling in the decoder but couldn't get it to work properly with the training loop.

In [3]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        #print("Input shape:", x.shape)
        x = self.encoder(x)
        #print("Shape after encoding:", x.shape)
        x = self.decoder(x)
        #print("Shape after decoding:", x.shape)
        return x


### Training Loop

The learning rate scheduler made a noticeable difference in bringing the training and value losses closer together.

In [4]:
# Initialize the model, loss, and optimizer
ae_model = Autoencoder()
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ae_model = ae_model.to(device)

criterion = nn.MSELoss()    
optimizer = optim.SGD(ae_model.parameters(), 
                      lr=0.010, 
                      momentum=0.9)

scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=10)
# Training loop for 50 epochs
N_EPOCHS = 50
tr_loss_hist = []
val_loss_hist = []
for epoch in range(N_EPOCHS):

    # Training 
    train_loss = 0.0
    ae_model.train() # <1>
    for batch in train_loader:
        images,_ = batch
        images = images.cuda()
        optimizer.zero_grad()

        outputs = ae_model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    # Validation
    val_loss = 0.0
    ae_model.eval() # <2>
    for batch in val_loader:
        images,_ = batch
        images = images.cuda()

        outputs = ae_model(images)
        loss = criterion(outputs, images)

        val_loss += loss.item()

    print("Epoch: {} Train Loss: {} Val Loss: {}".format(
                  epoch, 
                  train_loss/len(train_loader), 
                  val_loss/len(val_loader)))
    tr_loss_hist.append(train_loss/len(train_loader))
    val_loss_hist.append(val_loss/len(val_loader))

Epoch: 0 Train Loss: 0.8791889500617981 Val Loss: 0.8912959585064336
Epoch: 1 Train Loss: 0.8661818257967631 Val Loss: 0.8717702282102484
Epoch: 2 Train Loss: 0.8097550594806671 Val Loss: 0.6910032065291154
Epoch: 3 Train Loss: 0.5286659320195516 Val Loss: 0.4192412362286919
Epoch: 4 Train Loss: 0.38121739943822225 Val Loss: 0.33711929187962886
Epoch: 5 Train Loss: 0.3266071234146754 Val Loss: 0.3005913496017456
Epoch: 6 Train Loss: 0.2990867958466212 Val Loss: 0.2818088194257335
Epoch: 7 Train Loss: 0.28451534390449523 Val Loss: 0.2718357408517285
Epoch: 8 Train Loss: 0.27982172310352327 Val Loss: 0.26706628168099805
Epoch: 9 Train Loss: 0.27839208314816155 Val Loss: 0.2653751157616314
Epoch: 10 Train Loss: 0.27756725907325747 Val Loss: 0.2654864844915114
Epoch: 11 Train Loss: 0.2770154458284378 Val Loss: 0.2653449918878706
Epoch: 12 Train Loss: 0.2765861852963765 Val Loss: 0.2658187115662976
Epoch: 13 Train Loss: 0.27623687585194906 Val Loss: 0.2657736598661071
Epoch: 14 Train Loss: 

### Testing

In [5]:
# Initialize variables to store test losses
total_mse_loss = 0.0
total_mae_loss = 0.0

# Set the model to evaluation mode
ae_model.eval()

# Loop over test data
for x_test_batch, _ in test_loader:  # Note: we don't need labels for autoencoders

    # Move the test data to the same device as the model
    x_test_batch = x_test_batch.to(device)

    # Forward pass to get the model's reconstructed output
    reconstructed_batch = ae_model(x_test_batch)
    print("Shape of x_test_batch:", x_test_batch.shape)
    print("Shape of reconstructed_batch:", reconstructed_batch.shape)

    # Calculate the loss between the reconstructed and original data
    mse_loss = nn.MSELoss()(reconstructed_batch, x_test_batch)
    mae_loss = nn.L1Loss()(reconstructed_batch, x_test_batch)

    # Accumulate the loss
    total_mse_loss += mse_loss.item()
    total_mae_loss += mae_loss.item()

# Calculate the average loss
average_mse_loss = total_mse_loss / len(test_loader)
average_mae_loss = total_mae_loss / len(test_loader)

print(f'Average MSE Loss: {average_mse_loss}')
print(f'Average MAE Loss: {average_mae_loss}')


Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([16, 3, 200, 200])
Shape of x_test_batch: torch.Size([16, 3, 200, 200])
Shape of reconstructed_batch: torch.Size([1